# Summarize QA/QC Results 

Create summary tables from QAQC-PFRA notebook scraps in one or more folders on Amazon Web Services' S3.

### _Development Notebook_

### Import libraries

In [1]:
import sys;sys.path.append('../')
import gdal
from hecrasio.core import *
from hecrasio.qaqc import *
from hecrasio.s3tools import *
import scrapbook as sb
import pandas as pd
import shutil

### Set pandas display options

In [2]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 1000)

### Identify notebooks on S3

In [3]:
# Either of these will work

books = pull_scraps(single_folder = "s3://azavea/jwx_test/data/pfra_test_nbs")

# books = pull_scraps(multi_folder = 'yes', bucket = 'azavea', prefix = 'jwx_test/nbs/nb0', name_selector = 'DC')

### Create QA/QC results table

In [4]:
results = make_qaqc_table(books)
results.head(5)

WARNING! DC_F02_NBR_E0029_out had the following global errors: ["Check files...too many hdf's found", 'Failed to read RAS plan: rasPlan error', 'Failed to glue Plan Information', 'Failed to glue Plan Parameters', 'Failed to Show Results', 'Failed to glue Results Data', 'Failed to glue Plan Summary']
WARNING! DC_F02_NBR_E0097_out had the following global errors: ['Failed to Show Results', 'Failed to glue Results Data', 'Failed to glue Plan Summary']
WARNING! The following Notebooks had null 1D Cores values. These notebooks were dropped from the QA/QC table with the assumption these notebooks are bad: ['DC_F02_NBR_E0029_out']


,1D Cores,1D Methodology,1D2D Flow Tolerance,1D2D MaxIter,1D2D Minimum Flow Tolerance,1D2D WS Tolerance,2D Boundary Condition Ramp Up Fraction,2D Boundary Condition Volume Check,2D Cores,2D Coriolis,2D Eddy Viscosity Transverse Mixing Coefficient,2D Equation Set,2D Initial Conditions Ramp Up Time (hrs),2D Latitude for Coriolis,2D Maximum Iterations,2D Names,2D Number of Time Slices,2D Only,2D Theta,2D Theta Warmup,2D Volume Tolerance,2D Water Surface Tolerance,Base Output Interval,Computation Time Courant Method,Computation Time DSS,Computation Time Step Base,Computation Time Step Count To Double,Computation Time Step Max Courant,Computation Time Step Max Doubling,Computation Time Step Max Halving,Computation Time Step Min Courant,Computation Time Total,Flow Filename,Flow Title,Geometry Filename,Geometry Title,HDF Chunk Size,HDF Compression,HDF Fixed Rows,HDF Flush Buffer,HDF Spatial Parts,HDF Use Max Rows,HDF Write Face Node Velocities,HDF Write Time Slices,HDF Write Warmup,Instability Count,Max Velocity,Maximum WSEL Error,Pardiso Solver,Plan Filename,Plan Name,Plan ShortID,Plan Title,Run Time Window,Simulation End Time,Simulation Start Time,Solution,Time Solution Went Unstable,Time Stamp Solution Went Unstable,Time Window,Vol Accounting,Vol Accounting 2D Bdry,Vol Accounting Cell End,Vol Accounting Cell Start,Vol Accounting DS Outflow,Vol Accounting Error,Vol Accounting Error Percentage,Vol Accounting Groundwater,"Vol Accounting LS, Pump Div",Vol Accounting Lat Hydro,Vol Accounting US Inflow
ARLAMS,0,Finite Difference,0.1,0,1,0.01,"[0.1000000015, 0.1000000015, 0.1000000015, 0.1...","[False, False, False, False, False]",0,False,"[3.402823466e+38, 3.402823466e+38, 3.402823466...","[Dynamic Wave, Dynamic Wave, Dynamic Wave, Dyn...","[3.402823466e+38, 3.402823466e+38, 3.402823466...","[3.402823466e+38, 3.402823466e+38, 3.402823466...","[20, 20, 20, 20, 20]","[Area1, Area2, Area3, Area4, Area5]","[1, 1, 1, 1, 1]",True,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0099999998, 0.0099999998, 0.0099999998, 0.0...","[0.0099999998, 0.0099999998, 0.0099999998, 0.0...",1HOUR,NaN,00:00:00,30SEC,NaN,NaN,NaN,NaN,NaN,02:36:45,ARLAMS.u01,Precip_Excess,ARLAMS.g01,ARLAMS_Pluvial,1,1,1,False,1,0,False,False,False,0,13.1837,0,False,ARLAMS.p01,reducedPluvial_E005,reducedPluvial_E005,reducedPluvial_E005,04JAN2019 20:04:14 to 04JAN2019 22:40:54,05May2018 00:00:00,01May2018 00:00:00,Unsteady Finished Successfully,00:00:00,Not Applicable,01May2018 00:00:00 to 05May2018 00:00:00,Volume Accounting in Acre Feet,0,662727,0,0,854.456,0.119748,0,0,0,0
DC_F01_B01,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,[3.402823466e+38],[Dynamic Wave],[3.402823466e+38],[3.402823466e+38],[20],[D01],[1],True,[1.0],[1.0],[0.0099999998],[0.0099999998],15MIN,Representative Length/Velocity,00:00:00,15SEC,2,2,2,1,0.5,03:41:13,DC_F01_B01.u01,Extreme_event,DC_F01_B01.g01,D01,1,1,1,False,1,0,False,False,False,3477,96671.7,0,False,DC_F01_B01.p01,Extreme_event,Extreme_event,Extreme_event,21AUG2019 17:24:25 to 21AUG2019 21:05:37,05May1996 00:00:00,01May1996 00:00:00,Unsteady Finished Successfully,00:00:00,Not Applicable,01May1996 00:00:00 to 05May1996 00:00:00,Volume Accounting in Acre Feet,0,106923,87092.2,0,-95.1274,0.00205718,0,0,0,0
DC_F01_NBR_E0001_out,0,Finite Difference,0.1,0,1,0.01,[0.1000000015],[False],0,False,[3.402823466e+38],[Dynamic Wave],[3.402823466e+38],[3.402823466e+38],[20],[D01],[1],True,[1.0],[1.0],[0.0099999998],[0.0099999998],15MIN,Representative Length/Velocity,00:00:00,30SEC,2,2,2,1,0.5,03:55:48,DC_F01_NBR.u01,E0001,DC_F01_NBR.g01,D01,1,1,1,False,1,0,False,False,False,6824,37015.7,0,False,DC_F01_NBR.p01,test_scaling,test_scaling,test_scaling,15JUL2019 17:03:30 to 15JUL2019 20:59:17,05May2000 10:00:0,01May2000 10:00:0,Unsteady Finished Successfully,00:00:00,Not Applicable,01May2000 10:00:00 to 05May2000 10:00:0,Volume Accounting in Acre Feet,0,107467,84515.9,0,-1.9559,4.23324e-05,0,0,0,0
DC_F01_NBR_E0001_p01,0,Finite Difference

### Create unique values summary table
**Not implimented:** If length of unque value list is equal to a single value, ignore attribute and return that all values are the same.

In [5]:
relevant_columns = ['Base Output Interval',
                    '2D Equation Set',
                    '2D Volume Tolerance',
                    '2D Latitude for Coriolis',
                    '2D Water Surface Tolerance',
                    'Solution',
                    'Vol Accounting Error',
                    'Max Velocity',
                    'Instability Count']

unique_df = identify_unique_values(results, relevant_columns)

unique_df.head(10)

,Unique_Values
Result_Attribute,
Base Output Interval,"[15MIN, 1HOUR]"
2D Equation Set,[Dynamic Wave]
2D Volume Tolerance,[0.0099999998]
2D Latitude for Coriolis,[3.402823466e+38]
2D Water Surface Tolerance,[0.0099999998]
Solution,"[Unsteady Finished Successfully, nan]"
Vol Accounting Error,"[-95.1273574829, -1.9559047222, 0.5684285164, ..."
Max Velocity,"[13.1836738586, 40.2980041504, 430.2389526367,..."
Instability Count,"[0.0, 6.0, 123.0, 3477.0, 6824.0, nan, 0.0, 2...."


### Create threshold summary table

In [6]:
create_summary_table(unique_df, results)


Now evaluating: Solution

The following notebooks have alarming values for this attribute

Notebook                       Value (none)
-------------------------------------------------------------------------------
DC_F02_NBR_E0097_out           nan
-------------------------------------------------------------------------------

Now evaluating: Vol Accounting Error

The following notebooks have alarming values for this attribute

Notebook                       Value (none)
-------------------------------------------------------------------------------
ARLAMS                         854.4564819336
DC_F01_B01                     -95.1273574829
DC_F01_NBR_E0001_out           -1.9559047222
DC_F01_NBR_E0001_p01           -1.9559047222
DC_F01_NBR_E0004_out           36.8437576294
DC_F02_NBR_E0001_out           0.5684285164
P03_P04_100yr                  764.8336791992
Sacramento_P01_H06_E0001       18670.203125
atkinstest_P01_H00_p01         0.4258438349
------------------------------------

# END